+ 4 pts resample from data
+ 4 pts resampling done with replacement
+ 4 pts take several sets of resamplings
+ 2 pts implies repeated resampling, but does not mention explicitly
+ 4 pts determine the coeﬃcients from the line of best fit for each resampling
+ 2 pts mentions calculating a metric from each resampling, but does not specify OR metric calculated from each resampling, but incorrect one
+ 4 pts gather all lines of best fit to gauge reliability (plot all lines together or plot histogram of coeﬃcients)
+ 2 pts mentions comparing coeﬃcients to coeﬃcient for original data
+ 4 pts incorrect description of bootstrapping, but mentions generation of a confidence interval around line of best fit
+ 0 pts incorrect or no attempt

In [45]:
from ollama import chat
from ollama import ChatResponse

In [46]:
student_response = "Bootstrapping creates a confidence interval of possible lines of best fit. 1) Obtain a sample data set from observation/experiment with replacement. 2) Generate a new randomized data set from the sample data set (step 1) that has the same number of observations. ex: sample has 1000 points, new data set has 1000 based on this. 3) Repeat step 2 multiple times (>10) and determine the line of best fit for each scatter plot. 4) Place these lines on a scatter plot and a range of linear lines should be seen, this will determine if the original line is within the range of a reliable fit (confidence interval of linear model lines)"

rubric_component = "Implies repeated resampling, but does not mention explicitly"

context = "Describe, step-by-step, the process of bootstrapping to estimate the reliability of coefficients fit by a linear model."

In [47]:
response: ChatResponse = chat(model='gemma2', messages=[
  {
      'role': 'system',
      'content': f"You are a biology grading assistant. Your task is to evaluate student responses to determine if the rubric component is explicitly addressed{', taking into account the information provided to the student in the context section' if context else ''}, regardless of its factual accuracy. Do not use your own knowledge to judge the correctness of the rubric component or the student's response. Your role is to check if the student's response matches what the rubric asks for, even if the rubric component itself contains inaccuracies.",
  },
  {
      'role': 'user',
      'content': f"""
        The rubric component is: '{rubric_component}'.
        The student response is: '{student_response}'.
        {'The context is: ' + f"'{context}'" if context else ''}

        Your task is to evaluate if the student's response demonstrates understanding of the core concepts in the rubric component. Walk through your analysis step by step:

        1. ANALYSIS OF RUBRIC:
        - Break down the key concepts and requirements from the rubric component
        - What are the essential ideas that must be demonstrated?

        2. ANALYSIS OF RESPONSE:
        - Identify relevant portions of the student's response
        - Look for both direct statements and implied understanding
        - Consider alternative phrasings and terminology that convey the same concepts

        3. EVIDENCE EVALUATION:
        - Match concepts from the rubric to evidence in the response
        - Consider semantic equivalence and contextual meaning
        - Note any missing or misunderstood concepts
        - Assess if implied or indirect demonstrations of knowledge are sufficient

        4. FINAL DETERMINATION:
        After completing your analysis, provide your final evaluation of whether the rubric component is satisfied.

        Important evaluation guidelines:
        - Focus on conceptual understanding rather than exact wording matches
        - Students may use different terminology to express the same ideas
        - Consider the context of the field when evaluating semantic equivalence
        - Look for evidence of understanding rather than perfect articulation
        - If a concept is implied through a clear description of its implementation or consequences, consider it present

        Examples of semantic equivalence:
        - "Gather all lines" = "Compile all lines" = "Collect all lines" = "Put together all lines"
        - "Gauge reliability" = "Assess reliability" = "Evaluate reliability" = "Determine reliability"
        - "Plot together" = "Display together" = "Show together" = "Visualize together"

        Your response MUST:
        1. Show your complete thought process through steps 1-3
        2. Only provide your final determination (satisfied/unsatisfied) AFTER showing this analysis
        3. End with the word "EVALUATION:" followed by your clear Yes/No determination and brief summary of why

        """
  }
])

In [17]:
response_content = response.message.content

type(response_content)

str

In [42]:
grader_json_schema = """{
    "rubricComponentSatisfied": <true/false based on Yes/No>,
    "explanation": <concise summary of key reasoning>
}
"""

evaluator_json_schema = """{
    "gradersAgree": <true/false>
    "consensusEvaluation": <'Yes'/'No'/'No consensus reached'>
    "explanation": <concise summary of key reasoning>
"""

In [43]:
jsonified: ChatResponse = chat(model='gemma2', messages=[
    {
        'role': 'system',
        'content': f"""            
            You are a JSON formatter. Your task is to convert a detailed grading analysis into a standardized JSON response.

            The input will contain a complete analysis followed by a final evaluation. Your job is to:
            1. Identify the final Yes/No determination
            2. Extract the key reasoning from the analysis
            3. Format these into valid JSON

            Rules for JSON creation:
            1. Only include the final determination, not intermediate thoughts
            2. Summarize the key reasoning in 2-3 clear sentences
            3. Format the response exactly as:

            {grader_json_schema}

            Format requirements:

            - Use exact property names as shown
            - Use true/false (not "true"/"false" or Yes/No)
            - Keep explanation clear and concise
            - Ensure valid JSON syntax
            - Do not include any other fields
            - Do not include any text before or after the JSON

            """
    },
    {
        'role': 'user',
        'content': response_content
    }
])

In [43]:
jsonified_content = jsonified.message.content

print(jsonified_content)

NameError: name 'jsonified' is not defined

In [42]:
import json

loaded_json = json.loads(jsonified_content)

type(loaded_json)

print(loaded_json)

NameError: name 'jsonified_content' is not defined

In [34]:
from models.response_models.grader_response_model import GraderResponseModel
from models.response_models.evaluator_response_model import EvaluatorResponseModel

validation_model = GraderResponseModel(rubricComponentSatisfied=True,explanation="Test")
# validation_model = EvaluatorResponseModel(gradersAgree=True,consensusEvaluation="No",explanation="Test")

if type(validation_model) is GraderResponseModel:
    print("Grader")
elif type(validation_model) is EvaluatorResponseModel:
    print("Evaluator")
else:
    print("Error")

Grader


In [41]:
isinstance(validation_model, GraderResponseModel)

True

In [32]:
from models.prompt_models.query_model import QueryModel

import importlib
import utils.query_large_language_model
import utils.initiate_debate
importlib.reload(utils.query_large_language_model)
importlib.reload(utils.initiate_debate)
from utils.initiate_debate import initiate_debate

In [33]:
query = QueryModel(rubric_component="Implies repeated resampling, but does not mention explicitly",
                    student_response="Bootstrapping creates a confidence interval of possible lines of best fit. 1) Obtain a sample data set from observation/experiment with replacement. 2) Generate a new randomized data set from the sample data set (step 1) that has the same number of observations. ex: sample has 1000 points, new data set has 1000 based on this. 3) Repeat step 2 multiple times (>10) and determine the line of best fit for each scatter plot. 4) Place these lines on a scatter plot and a range of linear lines should be seen, this will determine if the original line is within the range of a reliable fit (confidence interval of linear model lines)",
                    context="Describe, step-by-step, the process of bootstrapping to estimate the reliability of coefficients fit by a linear model."
                    )

initiate_debate(query=query)

ic| f"Round {len(round_list) + 1} initiated.": 'Round 1 initiated.'


TypeError: 

In [ ]:
{
        "gradersAgree": True,
        "consensusEvaluation": "Yes",
        "explanation": "The student describes repeatedly generating new datasets and performing calculations on each, clearly implying bootstrapping despite not using the term. This fulfills the rubric's requirement of demonstrating the concept through description." 
}